In [1]:
import numpy as np
import pandas as pd
import import_ipynb

In [ ]:
import Orders, Freights
import Customers, PlantPorts, ProductsPerPlant, WhCapacities, WhCosts

# 2da Etapa: Optimización

*Función Objetivo*\
Se desea conocer un conjunto de plantas de almacenaje, puertos de origen y puertos de destino que minimicen el costo total de la cadena de suministro.

Nomenclatura:
- k: ID de pedido
- i: ID de planta de almacenamiento
- p: ID de puerto de origen
- j: ID de puerto de destino
- c: ID de transportista
- s: Nivel de servicio
- t: Tiempo de envío [días]
- m: Modo de transporte (aire o tierra)
- q: Cantidad de items en cada pedido [items]
- w: Peso [kg/item]
- F: Peso máximo [kg/item]

- P: Costo de almacenamiento [USD/item]
- M: Costo fijo de transporte [USD/kg]
- X: Costo de almacenamiento [USD]
- Y: Costo de transporte [USD]
- Z: Costo total de la cadena de suministro [USD]

### Costos del Almacenamiento



In [3]:
df1 = pd.merge(Orders.pedidos, WhCosts.costos_almacenaje, on = "Planta")
df1 = pd.merge(df1, PlantPorts.puertos_por_planta, on = ["Planta", "Puerto de origen"])
df1 = pd.merge(df1, ProductsPerPlant.productos_por_planta, on = ["Planta", "Producto"])
df1


,Pedido,Fecha,Puerto de origen,Transportista,TPT,Nivel de servicio,Envío adelantado [días],Envío tardío [días],Cliente,Producto,Planta,Puerto de destino,Items/pedido,Peso [kg/item],Costo [USD/item]
0,1447306622,2013-05-26,4,V44_3,4,CRF,0,0,V555555555555555_29,1699333,3,9,1529,3.008251,0.517502
1,1447306541,2013-05-26,4,V44_3,4,CRF,0,0,V555555555555555_29,1699268,3,9,1495,2.653360,0.517502
2,1447392840,2013-05-26,4,V44_3,4,CRF,0,0,V555555555555555_29,1699268,3,9,899,1.326680,0.517502
3,1447306543,2013-05-26,4,V44_3,4,CRF,0,0,V555555555555555_29,1699268,3,9,1437,2.653360,0.517502
4,1447184599,2013-05-26,4,V44_3,0,CRF,3,0,V55555555_32,1699268,3,9,1518,2.653360,0.517502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9037,1447199663,2013-05-26,4,V444_0,2,DTP,3,0,V55555555_5,1652883,13,9,358,40.038746,0.469707
9038,1447384590,2013-05-26,4,V444_0,2,DTP,0,0,V55555555_5,1652883,13,9,462,79.687104,0.469707
9039,1447345606,2013-05-26,4,V444_0,2,DTP,3,0,V55555555_5,1674378,13,9,277,7.307206,0.469707
9040,1447396669,2013-05-26,4,V444_0,2,DTP,3,0,V55555555_5,1674380,13,9,358,6.988704,0.469707


$$ X_{i} = \sum_{k}{(q_{k}*P_i)} $$

In [4]:
df1["Costo [USD/pedido]"] = df1["Items/pedido"] * df1["Costo [USD/item]"]

g1 = df1.groupby(by = ["Planta"]).agg({"Costo [USD/pedido]": np.sum})
g1

,Costo [USD/pedido]
Planta,
3,1.480657e+07
4,1.491191e+02
8,1.448357e+05
9,1.040969e+05
12,8.659076e+04
13,2.037542e+04


### Capacidad del Almacenamiento

$$ \sum_{k}{o_{ki}} $$

In [5]:
g2 = df1.groupby(by = ["Planta"]).agg({"Pedido": np.size}).rename(columns = {"Pedido": "Recuento de Pedidos"})
g2

,Recuento de Pedidos
Planta,
3,8541
4,1
8,102
9,12
12,300
13,86


$$ \sum_{k}{o_{ki}} \leq \ C_{i} $$

In [6]:
df2 = pd.merge(g2, WhCapacities.capacidad_almacenaje, on = "Planta")
df2["Recuento de Pedidos"] = np.where(df2["Recuento de Pedidos"] <= df2["Capacidad [pedidos/día]"], df2["Recuento de Pedidos"], df2["Capacidad [pedidos/día]"])
df2

,Planta,Recuento de Pedidos,Capacidad [pedidos/día]
0,3,1013,1013
1,4,1,554
2,8,14,14
3,9,11,11
4,12,209,209
5,13,86,490


### Costos de Transporte

In [7]:
df3 = pd.merge(Orders.pedidos, Freights.fletes, on = ["Puerto de origen", "Puerto de destino", "Transportista", "Nivel de servicio"])
df3 = pd.merge(df3, PlantPorts.puertos_por_planta, on = ["Planta", "Puerto de origen"])
df3 = pd.merge(df3, ProductsPerPlant.productos_por_planta, on = ["Planta", "Producto"])
df3

,Pedido,Fecha,Puerto de origen,Transportista,TPT,Nivel de servicio,Envío adelantado [días],Envío tardío [días],Cliente,Producto,...,Puerto de destino,Items/pedido,Peso [kg/item],Peso mínimo [kg/item],Peso máximo [kg/item],Costo mínimo [USD/kg],Tasa de flete [USD/kg],Modo de transporte,Envío [días],Tipo de transportista
0,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,9,263,2.543556,2000.0,99999.99,1.4992,0.0424,Aire,3,V88888888_0
1,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,9,263,2.543556,100.0,249.99,1.4992,0.0424,Aire,3,V88888888_0
2,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,9,263,2.543556,250.0,499.99,1.4992,0.0424,Aire,3,V88888888_0
3,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,9,263,2.543556,500.0,1999.99,1.4992,0.0424,Aire,3,V88888888_0
4,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,9,263,2.543556,0.0,99.99,1.4992,0.0484,Aire,3,V88888888_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146175,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,9,1086,2.538912,2000.0,99999.99,1.4992,0.0424,Aire,2,V88888888_0
146176,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,9,1086,2.538912,100.0,249.99,1.4992,0.0424,Aire,2,V88888888_0
146177,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,9,1086,2.538912,250.0,499.99,1.4992,0.0424,Aire,2,V88888888_0
146178,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,9,1086,2.538912,500.0,1999.99,1.4992,0.0424,Aire,2,V88888888_0


$$ \sum_{k}{w_{kpjcstm}} $$

In [8]:
g3 = df3.groupby(by = ["Puerto de origen", "Puerto de destino", "Transportista", 
                       "Nivel de servicio", "Envío [días]", "Modo de transporte"]).agg({"Peso [kg/item]": np.sum}).rename(columns = {"Peso [kg/item]": "Suma de Peso [kg/item]"})

g3.round(2)

Suma de Peso [kg/item]
Puerto de origen Puerto de destino Transportista Nivel de servicio Envío [días] Modo de transporte                        
4                9                 V444_0        DTD               2            Aire                               5854.40
                                                                   3            Aire                               5854.40
                                                 DTP               2            Aire                             578581.42
                                                                   3            Aire                             578581.42
                                   V444_1        DTD               1            Aire                             531490.03
                                                                   2            Aire                             531490.03

$$ \sum_{k}{w_{kpjcstm}} \leq max \ F_{kpjcstm} $$

In [9]:
df3["Suma de Peso [kg/item]"] = df3.groupby(["Puerto de origen", "Puerto de destino", "Transportista", 
                       "Nivel de servicio", "Envío [días]", "Modo de transporte"])["Peso [kg/item]"].transform("sum")

df3["Suma de Peso [kg/item]"] = np.where(df3["Suma de Peso [kg/item]"] <= df3["Peso máximo [kg/item]"], df3["Suma de Peso [kg/item]"], df3["Peso máximo [kg/item]"])
df3

,Pedido,Fecha,Puerto de origen,Transportista,TPT,Nivel de servicio,Envío adelantado [días],Envío tardío [días],Cliente,Producto,...,Items/pedido,Peso [kg/item],Peso mínimo [kg/item],Peso máximo [kg/item],Costo mínimo [USD/kg],Tasa de flete [USD/kg],Modo de transporte,Envío [días],Tipo de transportista,Suma de Peso [kg/item]
0,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,263,2.543556,2000.0,99999.99,1.4992,0.0424,Aire,3,V88888888_0,99999.99
1,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,263,2.543556,100.0,249.99,1.4992,0.0424,Aire,3,V88888888_0,249.99
2,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,263,2.543556,250.0,499.99,1.4992,0.0424,Aire,3,V88888888_0,499.99
3,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,263,2.543556,500.0,1999.99,1.4992,0.0424,Aire,3,V88888888_0,1999.99
4,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,263,2.543556,0.0,99.99,1.4992,0.0484,Aire,3,V88888888_0,99.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146175,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,1086,2.538912,2000.0,99999.99,1.4992,0.0424,Aire,2,V88888888_0,99999.99
146176,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,1086,2.538912,100.0,249.99,1.4992,0.0424,Aire,2,V88888888_0,249.99
146177,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,1086,2.538912,250.0,499.99,1.4992,0.0424,Aire,2,V88888888_0,499.99
146178,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,1086,2.538912,500.0,1999.99,1.4992,0.0424,Aire,2,V88888888_0,1999.99


$$ Si \ s = CRF \ \Rightarrow \ Y_{kpjcstm} = 0 $$

$$ Si \ s \neq CRF, \ m = Tierra \ \Rightarrow \ Y_{kpjcstm} = \frac{w_{kpjcstm}}{\sum_{k}{w_{kpjcstm}}}*R_{kpjcstm} $$

$$ Si \ s \neq CRF, \ m \neq Tierra \ \Rightarrow \ Y_{kpjcstm} = w_{kpjcstm}*R_{kpjcstm} $$

$$ Si \ Y_{kpjcstm} < M_{kpjcstm} \ \Rightarrow \ Y_{kpjcstm} = M_{kpjcstm} $$

In [16]:
a = df3["Peso [kg/item]"] / df3["Suma de Peso [kg/item]"] * df3["Tasa de flete [USD/kg]"]
b = df3["Peso [kg/item]"] * df3["Tasa de flete [USD/kg]"]
c = df3["Costo mínimo [USD/kg]"]

df3["Costo de transporte [USD]"] = np.where(df3["Nivel de servicio"] == "CRF", 0, 
                                            np.where(df3["Modo de transporte"] == "Tierra", np.where(a < c, a, c), np.where(b < c, b, c)))

df3

,Pedido,Fecha,Puerto de origen,Transportista,TPT,Nivel de servicio,Envío adelantado [días],Envío tardío [días],Cliente,Producto,...,Peso [kg/item],Peso mínimo [kg/item],Peso máximo [kg/item],Costo mínimo [USD/kg],Tasa de flete [USD/kg],Modo de transporte,Envío [días],Tipo de transportista,Suma de Peso [kg/item],Costo de transporte [USD]
0,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,2.543556,2000.0,99999.99,1.4992,0.0424,Aire,3,V88888888_0,99999.99,0.11
1,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,2.543556,100.0,249.99,1.4992,0.0424,Aire,3,V88888888_0,249.99,0.11
2,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,2.543556,250.0,499.99,1.4992,0.0424,Aire,3,V88888888_0,499.99,0.11
3,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,2.543556,500.0,1999.99,1.4992,0.0424,Aire,3,V88888888_0,1999.99,0.11
4,1447164685,2013-05-26,4,V444_0,2,DTP,0,0,V555555555555555_29,1680246,...,2.543556,0.0,99.99,1.4992,0.0484,Aire,3,V88888888_0,99.99,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146175,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,2.538912,2000.0,99999.99,1.4992,0.0424,Aire,2,V88888888_0,99999.99,0.11
146176,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,2.538912,100.0,249.99,1.4992,0.0424,Aire,2,V88888888_0,249.99,0.11
146177,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,2.538912,250.0,499.99,1.4992,0.0424,Aire,2,V88888888_0,499.99,0.11
146178,1447382863,2013-05-26,4,V444_0,2,DTP,3,0,V555555_6,1681882,...,2.538912,500.0,1999.99,1.4992,0.0424,Aire,2,V88888888_0,1999.99,0.11


$$ Y_{pj} = \sum_{cstm}{Y_{pjcstm}} $$

In [15]:
g3 = df3.groupby(by = ["Puerto de origen", "Puerto de destino"]).agg({"Costo de transporte [USD]": np.sum}).round(2)
g3

,,Costo de transporte [USD]
Puerto de origen,Puerto de destino,
4,9,111527.17


### Costos Totales

$$ Z_{ipj} = min \ (X_{i} + Y_{pj}) $$

In [12]:
g4 = df3.groupby(by = ["Planta", "Puerto de origen", "Puerto de destino"]).groups
chain = pd.DataFrame(data = g4.keys(), columns = ["Planta", "Puerto de origen", "Puerto de destino"])
chain

,Planta,Puerto de origen,Puerto de destino
0,3,4,9
1,8,4,9
2,9,4,9
3,12,4,9
4,13,4,9


*¿Qué conjunto Planta, Puerto de origen y Puerto de destino tiene la menor Suma de Costo Total?*

In [17]:
cost = g1["Costo [USD/pedido]"].values + g3["Costo de transporte [USD]"].values
cost.round(2)

array([14918093.09,   111676.29,   256362.82,   215624.03,   198117.93,
         131902.59])